In [ ]:
import sagemaker
from sagemaker.xgboost import XGBoostModel
from sagemaker.serverless import ServerlessInferenceConfig
import boto3 

# Cell 1: Setup
session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = session.boto_region_name
print(f"Using SageMaker role: {role}")

# Cell 2: Configuration
best_model_s3_uri = "s3://sagemaker-us-east-1-[ACCOUNT]/[TRAINING-MODEL]/output/model.tar.gz" 

# --- Define entry point script name ---
entry_script = "inference.py"

# --- Framework and Python versions ---
framework_version = "1.2-1"
python_version = "py3"

print(f"Using model artifact: {best_model_s3_uri}")

# Cell 3: Create SageMaker Model Object
xgb_model = XGBoostModel(
    model_data=best_model_s3_uri,
    role=role,
    entry_point=entry_script,
    framework_version='1.7-1',
    sagemaker_session=session
)

# Cell 4: Configure Serverless Inference
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=3072, 
    max_concurrency=5,      # Max concurrent requests handled by one instance
)

# Cell 5: Deploy the Model
endpoint_name = f"stock-rf-predict-serverless-{sagemaker.utils.unique_name_from_base('')}"
print(f"Deploying model to serverless endpoint: {endpoint_name}...")

predictor = xgb_model.deploy(
    serverless_inference_config=serverless_config,
    endpoint_name=endpoint_name,
    wait=True
)
print(f"\n Endpoint {endpoint_name} deployed successfully.")
print(f"Endpoint name: {predictor.endpoint_name}") 

# Cell 6 (Optional): Test Endpoint Directly from Notebook
sample_feature_payload = {
    "features": {
        'ret_1d': 0.005, 'mom_5d': 0.012, 'rsi_14': 55.0, 'abn_volume': 1.1,
        'ret_lag_1d': -0.002, 'volatility_20d': 0.015, 'ma_50d': 150.0, 'ma_200d': 140.0,
        'ma_trend_signal': 1, 'day_of_week': 2, 'month_of_year': 10, 'return_x_volume': 0.0055,
        'avg_sentiment_24h': 0.1, 'news_count_24h': 15, 'positive_count_24h': 5,
        'negative_count_24h': 2, 'sentiment_std_24h': 0.5, 'avg_sentiment_3d': 0.05
    }
}

In [19]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

json_predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=session,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

print("\nTesting endpoint with sample data...")
try:
    result = json_predictor.predict(sample_feature_payload)
    print(" Test Prediction Result:")
    print(result)
except Exception as e:
    print(f" Endpoint test failed: {e}")


Testing endpoint with sample data...
✅ Test Prediction Result:
{'prediction': 'Down', 'confidence': 0.5004833936691284, 'probability_up': 0.4995166063308716}


In [ ]:
import sagemaker
from sagemaker.xgboost import XGBoostModel 
from sagemaker.serverless import ServerlessInferenceConfig
import boto3

# Cell 1: Setup
session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = session.boto_region_name
print(f"Using SageMaker role: {role}")

# Cell 2: Configuration
best_xgb_model_s3_uri = "s3://sagemaker-us-east-1-[ACCOUNT]/[TRAINING-MODEL]/output/model.tar.gz" 

# --- Define entry point script name ---
entry_script = "inference.py" 

# --- Framework version ---
xgboost_framework_version = "1.7-1" 

print(f"Using XGBoost model artifact: {best_xgb_model_s3_uri}")
print(f"Using XGBoost framework version: {xgboost_framework_version}")

# Cell 3: Create SageMaker XGBoostModel Object
xgb_model = XGBoostModel(
    model_data=best_xgb_model_s3_uri,
    role=role,
    entry_point=entry_script,
    framework_version=xgboost_framework_version,
    sagemaker_session=session
)

# Cell 4: Configure Serverless Inference
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=2048, 
    max_concurrency=5,
)

# Cell 5: Deploy the Model
endpoint_name = f"stock-xgb-predict-serverless-{sagemaker.utils.unique_name_from_base('')}"
print(f"Deploying XGBoost model to serverless endpoint: {endpoint_name}...")

# Use the xgb_model object for deployment
predictor = xgb_model.deploy(
    serverless_inference_config=serverless_config,
    endpoint_name=endpoint_name,
    wait=True
)
print(f"\n Endpoint {endpoint_name} deployed successfully.")
print(f"Endpoint name: {predictor.endpoint_name}")

# Cell 6: Test Endpoint Directly from Notebook
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

# Sample payload 
sample_feature_payload = {
    "features": {
        'ret_1d': 0.005, 'mom_5d': 0.012, 'rsi_14': 55.0, 'abn_volume': 1.1,
        'ret_lag_1d': -0.002, 'volatility_20d': 0.015, 'ma_50d': 150.0, 'ma_200d': 140.0,
        'ma_trend_signal': 1, 'day_of_week': 2, 'month_of_year': 10, 'return_x_volume': 0.0055,
        'avg_sentiment_24h': 0.1, 'news_count_24h': 15, 'positive_count_24h': 5,
        'negative_count_24h': 2, 'sentiment_std_24h': 0.5, 'avg_sentiment_3d': 0.05
    }
}

json_predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=session,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

print("\nTesting endpoint with sample data...")
try:
    result = json_predictor.predict(sample_feature_payload)
    print(" Test Prediction Result:")
    print(result)
except Exception as e:
    print(f" Endpoint test failed: {e}")